In [9]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import warnings
np.seterr( over='ignore' )

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

class layers:
    def __init__(self,neu,r,c):
        self.neu=int(neu)
        self.wt=np.random.randn(int(r),int(c))*100
        self.bias=np.random.randn()

class network:
    def __init__(self,data_in,data_out,layers_list,l_rate=0.001):
        self.xt=np.array(data_in)
        self.yt=np.array(data_out)    
        self.l=int(len(li))
        self.ly=layers_list
        self.l_rate= float(l_rate)
        try:
            self.n=self.xt.shape[1]
        except:
            self.n=self.xt.shape[0]
        self.m=int(self.n/100)
        self.x_split=np.array_split(self.xt.T,self.m)
        self.y_split=np.array_split(self.yt.T,self.m)
     
    def sigm(self,x):
        
        return 1/(1+np.e**-x)


    def der_sigm(self,x):
        return self.sigm(x)*(1-self.sigm(x))

    def forward(self):    
        for i in range(self.l):
            if i==0:
                self.ly[i].z=np.dot(self.ly[i].wt,self.xt) +self.ly[i].bias              
            else:
                self.ly[i].z=np.dot(self.ly[i].wt,self.ly[i-1].a) + self.ly[i].bias
            self.ly[i].a=self.sigm(self.ly[i].z)
    
    def cost(self):
        return np.sum((np.dot(self.yt,np.log(self.ly[-1].a).T))+ np.dot( 1-self.yt,np.log(1-self.ly[-1].a).T ))
    
    def backprop(self):
        for i in range(self.l-1, -1, -1):
            if i!=self.l-1:
                self.ly[i].error=np.dot(self.ly[i+1].wt.T, self.ly[i+1].error)
            else:
                self.ly[i].error=(self.ly[i].a-self.yt)*self.ly[i].a*(1-self.ly[i].a)
    
    def update(self):
        for i in range(self.l-1, -1, -1):
            if i!=0:
                temp= np.dot(self.ly[i].error, self.ly[i-1].a.T)
            else:
                temp= np.dot(self.ly[i].error,self.xt.T)            
            temp_bias= np.mean(self.ly[i].error)
            self.ly[i].wt=self.ly[i].wt-temp*self.l_rate
            self.ly[i].bias=self.ly[i].bias-temp_bias*self.l_rate

    def train(self,epoch):
        for j in range(int(epoch)):
            self.forward()
            self.backprop()
            self.update()

        

    def implement(self,x_data):            # just one sample at a time
        x_data=np.array(x_data)/100
        l=int(x_data.shape[0])
        x_data=np.expand_dims(x_data,0)
        x_data=x_data.T
        self.xi=x_data
        for i in range(self.l):
            if i==0:
                self.ly[i].z=np.dot(self.ly[i].wt,x_data)+self.ly[i].bias
                self.ly[i].a=self.sigm( self.ly[i].z)
            else:
                self.ly[i].z=np.dot(self.ly[i].wt,self.ly[i-1].a)+self.ly[i].bias
                self.ly[i].a=self.sigm(self.ly[i].z)
        print(self.ly[-1].a)

    def train2(self):
        self.forward()
        self.backprop()
        self.update()
        while np.mean(np.abs(self.ly[-1].error))>(0.3):
            self.forward()
            self.backprop()
            self.update()

    def forward_cv(self):
        for i in range(self.l):
            self.ly[i].z_cv=np.empty((self.ly[i].neu,self.xc.shape[1]),dtype=np.float64)
            self.ly[i].a_cv=np.empty((self.ly[i].neu,self.xc.shape[1]),dtype=np.float64)

        for i in range(self.l):
            if i==0:
                for j in range(int(self.ly[i].neu)):
                    self.ly[i].z_cv[j,:]=np.dot(self.ly[i].wt,self.xc)[j,:] +self.ly[i].bias[j]              
            else:
                for j in range(int(self.ly[i].neu)):
                    self.ly[i].z_cv[j,:]=np.dot(self.ly[i].wt,self.ly[i-1].a_cv)[j,:] + self.ly[i].bias[j]
            self.ly[i].a_cv=self.sigm(self.ly[i].z_cv)
    def error_cv(self):
        return np.mean(np.square(self.yc- self.ly[-1].a_cv))

In [10]:

data=pd.read_csv("mnist_train_small.csv",header=None)
data.dropna()
d=np.array(data)
y=d[:,[0]]
x=np.delete(d,0,1)/255
z=np.eye(10)
yy=np.empty((y.shape[0],10),dtype=np.float64)
for i in range(y.shape[0]):
    for j in range(10):
        if y[i]==j:
            yy[i,:]=z[j,:]
        else:
            pass

In [30]:
li=[]
li.append(layers(10,10,784))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))


In [12]:
model=network(x.T,yy.T,li)


In [5]:
model.forward()

In [ ]:

while np.mean(np.abs(model.ly[-1].error))>=.09:
     model.train(50)

In [13]:
model.forward()
model.backprop()
model.update()

In [14]:
np.mean(np.abs(model.ly[-1].error))

0.006496815727406602

In [17]:
model.ly[-5].a #after training a lot

array([[1.42827613e-002, 1.62044884e-034, 1.62044884e-034, ...,
        1.62044884e-034, 1.00000000e+000, 1.00000000e+000],
       [5.69347505e-151, 8.57059532e-127, 8.57059532e-127, ...,
        8.57059532e-127, 2.12318274e-096, 2.12318275e-096],
       [1.00000000e+000, 1.00000000e+000, 1.00000000e+000, ...,
        1.00000000e+000, 1.00000000e+000, 1.00000000e+000],
       ...,
       [3.38247321e-065, 3.37800513e-096, 3.37800513e-096, ...,
        3.37800513e-096, 1.26044735e-043, 1.26044735e-043],
       [1.00000000e+000, 1.00000000e+000, 1.00000000e+000, ...,
        1.00000000e+000, 9.97636764e-001, 9.97636764e-001],
       [1.73739538e-120, 1.49475031e-010, 1.49475031e-010, ...,
        1.49475031e-010, 1.82787728e-095, 1.82787727e-095]])

In [15]:
np.mean(np.abs(model.ly[-1].error))

0.09437041450903498

In [35]:
model.forward()

In [36]:
model.ly[-1].a

array([[0.0981 , 0.0981 , 0.0981 , ..., 0.0981 , 0.0981 , 0.0981 ],
       [0.11215, 0.11215, 0.11215, ..., 0.11215, 0.11215, 0.11215],
       [0.09945, 0.09945, 0.09945, ..., 0.09945, 0.09945, 0.09945],
       ...,
       [0.1063 , 0.1063 , 0.1063 , ..., 0.1063 , 0.1063 , 0.1063 ],
       [0.0956 , 0.0956 , 0.0956 , ..., 0.0956 , 0.0956 , 0.0956 ],
       [0.10115, 0.10115, 0.10115, ..., 0.10115, 0.10115, 0.10115]])

In [27]:
def new(data):
    x= data.shape[0]
    for i in range(x):
        ind= np.argmax(data[i])
        data[i,:]=0
        data[i, ind]=1
    return data

In [28]:
data2=new(model.ly[-1].a.T)

In [29]:
data2

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])